# blender_utils

参考: [zju3dv/pvnet-rendering](https://github.com/zju3dv/pvnet-rendering/blob/master/base_utils.py)

In [1]:
import os
import sys
sys.path.append('..')
sys.path.append('../../')

import numpy as np

from src.config.config import cfg
from src.datasets.LineMod.LineModDB import LineModDB
from src.utils.base_utils import read_ply_model

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

## Setup

In [3]:
pvnet_linemod_dir = cfg.PVNET_LINEMOD_DIR
linemod_dir = cfg.LINEMOD_DIR
obj_name = 'ape'

## 1. ModelAligner Test

In [4]:
from blender_utils import ModelAligner

### インスタンス生成

In [5]:
aligner = ModelAligner(obj_name=obj_name,
                                                pvnet_linemod_dir=pvnet_linemod_dir,
                                                linemod_dir=linemod_dir)

### 1.1. get_translation_transform Test

In [6]:
translation_transform = aligner.get_translation_transform()

print('translation_transform type')
print(type(translation_transform))
print('translation_transform =')
print(translation_transform)

translation_transform type
<class 'numpy.ndarray'>
translation_transform =
[ 0.00642697  0.00273947 -0.0337945 ]


In [7]:
## 2. 

In [8]:
class PoseTransformer(object):
    rotation_transform = np.array([[1., 0., 0.],
                                                                     [0., -1., 0.],
                                                                     [0., 0., -1.]])
    translation_transforms = {}
    obj_name_to_number = {
        'ape': '001',
        'can': '004',
        'cat': '005',
        'driller': '006',
        'duck': '007',
        'eggbox': '008',
        'glue': '009',
        'holepuncher': '010'
    }
    blender_models={}

    def __init__(self,
                              obj_name: str,
                              pvnet_linemod_dir: str,
                              linemod_dir: str):
        """
        PoseTransformer の初期化関数

        Args:
            obj_name(str): LineMod データセットに含まれるオブジェクト名
            pvnet_linemod_dir(str): PVNet_LineMod データセットのディレクトリパス
            linemod_dir(str): LineMod データセットのディレクトリパス
        """
        self.obj_name = obj_name
        self.blender_model_path = os.path.join(pvnet_linemod_dir, '{}/{}.ply'.format(obj_name, obj_name))
        self.orig_model_path = os.path.join(linemod_dir, '{}/mesh.ply'.format(obj_name))
        self.model_aligner = ModelAligner(obj_name,
                                                                                pvnet_linemod_dir,
                                                                                linemod_dir)


    def orig_pose_to_blender_pose(self, pose: np.matrix) -> np.matrix:
        """
        オリジナルの LineMod データセットのオブジェクトの姿勢から Blender 上 でのオブジェクトの姿勢に変換するための関数

        Arg:
            pose(np.matrix): オリジナルの LineMod データセットのオブジェクトの姿勢情報([R|t] 行列)

        Return:
            (np.matrix): Blender 上 でのオブジェクトの姿勢情報([R|t] 行列)
        """
        rot, tra = pose[:, :3], pose[:, 3]
        tra = tra + np.dot(rot, self.model_aligner.get_translation_transform())
        rot = np.dot(rot, self.rotation_transform)
        return np.concatenate([rot, np.reshape(tra, newshape=[3, 1])], axis=-1)


    


    def orig_pose_to_blender_euler(self, pose):
        blender_pose = self.orig_pose_to_blender_pose(pose)
        return self.blender_pose_to_blender_euler(blender_pose)

## 2. PoseTransformer

In [9]:
from blender_utils import PoseTransformer

### 2.1. インスタンス生成

In [10]:
pose_transformer = PoseTransformer(obj_name,
                                                                               pvnet_linemod_dir,
                                                                               linemod_dir)

### 2.2. orig_pose_to_blender_pose Test

In [11]:
from src.datasets.LineMod.LineModDB import read_pose

In [12]:
img_num = 0
rt_dir = os.path.join(linemod_dir,
                                          obj_name,
                                          'data')
pose = read_pose(os.path.join(rt_dir, 'rot{}.rot'.format(img_num)),
                                      os.path.join(rt_dir, 'tra{}.tra'.format(img_num)))
print('pose =', pose)

RT = pose_transformer.orig_pose_to_blender_pose(pose).astype(np.float32)
print('RT =', RT)
#print(RT)

pose = [[-0.0963063  0.994044  -0.0510079 -0.111814 ]
 [-0.573321  -0.0135081  0.81922   -0.0783622]
 [ 0.813651   0.10814    0.571207   1.03612  ]]
RT = [[-0.0963063  -0.994044    0.0510079  -0.10798602]
 [-0.573321    0.0135081  -0.81922    -0.10976905]
 [ 0.813651   -0.10814    -0.571207    1.0223418 ]]


## 2.3. orig_pose_to_blender_euler Test

In [13]:
img_num = 0
rt_dir = os.path.join(linemod_dir,
                                          obj_name,
                                          'data')
pose = read_pose(os.path.join(rt_dir, 'rot{}.rot'.format(img_num)),
                                      os.path.join(rt_dir, 'tra{}.tra'.format(img_num)))
print('pose =', pose)

euler = pose_transformer.orig_pose_to_blender_euler(pose)
print('euler =', )

pose = [[-0.0963063  0.994044  -0.0510079 -0.111814 ]
 [-0.573321  -0.0135081  0.81922   -0.0783622]
 [ 0.813651   0.10814    0.571207   1.03612  ]]


NameError: name 'mat2euler' is not defined